In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

In [4]:
train, test = tf.keras.datasets.cifar10.load_data()

In [5]:
x_train = train[0]
y_train = train[1]
x_test = test[0]
y_test = test[1]

In [12]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
    
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
#     mean, variance = tf.nn.moments(input_layer, axes=[0,1,2], shift=None, name=None, keep_dims=False)
#     input_layer = tf.nn.batch_normalization(input_layer, mean, variance, variance_epsilon=1e-4, offset=0, scale=1.0)

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu,
      kernel_regularizer=regularizer)
    
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu,
      kernel_regularizer=regularizer)
        
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu, kernel_regularizer=regularizer)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10, kernel_regularizer=regularizer)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    loss = loss + tf.reduce_sum(tf.losses.get_regularization_losses())
    
#     print(tf.reduce_sum(tf.losses.get_regularization_losses()))

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [13]:
# Create the Estimator
cifar10_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./cifar10_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './cifar10_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000F408CC9E8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)

In [15]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train.astype(np.float32)},
    y=y_train.astype(np.int32),
    batch_size=100,
    num_epochs=None,
    shuffle=True)
cifar10_classifier.train(
    input_fn=train_input_fn,
    steps=1000,
    hooks=[])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./cifar10_convnet_model\model.ckpt-34702
INFO:tensorflow:Saving checkpoints for 34703 into ./cifar10_convnet_model\model.ckpt.
INFO:tensorflow:loss = 43.3844, step = 34703
INFO:tensorflow:global_step/sec: 2.32214
INFO:tensorflow:loss = 37.8505, step = 34803 (43.064 sec)
INFO:tensorflow:global_step/sec: 2.21446
INFO:tensorflow:loss = 31.2086, step = 34903 (45.158 sec)
INFO:tensorflow:global_step/sec: 2.20344
INFO:tensorflow:loss = 26.0542, step = 35003 (45.384 sec)
INFO:tensorflow:global_step/sec: 2.09392
INFO:tensorflow:loss = 21.3044, step = 35103 (47.757 sec)
INFO:tensorflow:global_step/sec: 1.87216
INFO:tensorflow:loss = 17.9161, step = 35203 (53.417 sec)
INFO:tensorflow:global_step/sec: 1.96455
INFO:tensorflow:loss = 14.918, step = 35303 (50.915 sec)
INFO:tensorflow:global_step/sec: 1.9844
INFO:tensorflow:loss = 12.3148, step = 35403 (50.377 sec)
INFO:tensorflow:global_step/sec: 2.00289
INFO:tenso

In [16]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test.astype(np.float32)},
    y=y_test.astype(np.int32),
    num_epochs=1,
    shuffle=False)
eval_results = cifar10_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-02-25-23:16:22
INFO:tensorflow:Restoring parameters from ./cifar10_convnet_model\model.ckpt-35702
INFO:tensorflow:Finished evaluation at 2018-02-25-23:16:39
INFO:tensorflow:Saving dict for global step 35702: accuracy = 0.4862, global_step = 35702, loss = 7.55584
{'accuracy': 0.4862, 'loss': 7.555841, 'global_step': 35702}


In [95]:
# Evaluate the model and print results
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test.astype(np.float32)},
    num_epochs=1,
    shuffle=False)
pred_results = cifar10_classifier.predict(input_fn=pred_input_fn)
# pred_probs = cifar10_classifier.predict(input_fn=pred_input_fn, predict_keys="probabilities")

In [96]:
pred_df = pd.DataFrame.from_records(list(pred_results))
pred_probs = pred_df['probabilities']
pred_df = pred_df.drop('probabilities', axis=1)
pred_df.index.names=['ID']
pred_df.columns = ['CLASS']
pred_df.to_csv('preds.csv')

INFO:tensorflow:Restoring parameters from ./cifar10_convnet_model - stable\model.ckpt-34702


In [97]:
pred_probs_array = np.asarray(pred_probs.tolist())
np.save('ans-uni', pred_probs_array.T)